## Local development
first draft of the main get_data function and its suporting functions that aim to return a one line df of collected data form the Purple Air sensor via its API

In [ ]:
pip install requests

In [ ]:
import requests
import os
import json
import datetime

In [ ]:
SENSOR_INDEX = YOUR_OWN_INFO
BASE_URL = YOUR_OWN_INFO
API_KEY = YOUR_OWN_INFO

In [ ]:
endpoint = f"/sensors/{SENSOR_INDEX}/"
url = f"{BASE_URL}{endpoint}"

headers = {
  "X-API-Key": API_KEY,
  "Accept": "application/json" # Good practice to explicitly request JSON
    }
params = {
  "fields": "last_seen,rssi,latitude,longitude,humidity,temperature,pressure,pm2.5,pm2.5_alt,visual_range, pm2.5_6hour" # Request specific fields
    }


In [ ]:
def get_data(url, headers, params):
  response = requests.get(url, headers=headers, params=params)
  data = response.json()

  now = datetime.datetime.now()
  now_unix = int(now.timestamp())

  return data, now_unix

In [ ]:
# Define the flatten_dict function
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


In [ ]:
def check_sensor_freshness(data, now_unix):
  delay_range=600 #10 minutes in seconds

  if not data:
    print("no data recived")
    return False
  if not data["sensor"]['last_seen']:
    print("Can not determine freshness of data as no sensor last seen data recieved")
    return False

  if data["sensor"]['last_seen']>now_unix - delay_range:
    print(f"Sensor is working in given delay range {delay_range} seconds")
    return True
  else:
    print("sensor is stal")
    return False

In [ ]:
data, now_unix = get_data(url, headers, params)
check_sensor_freshness(data, now_unix)

Sensor is working in given delay range 600 seconds


True

In [ ]:
## check validity of Enviromental data
def check_data_quality(data):

  if not (isinstance(data["sensor_temperature"], (int, float)) and 0 <= data["sensor_temperature"] <= 120):
    print(f"-Sensor {data['sensor_sensor_index']}- Invalid temperature value {data['sensor_temperature']}. Seting Temperatur to None.")
    data["sensor_temperature"]= None

  if not (isinstance(data["sensor_humidity"], (int, float)) and 0 <= data["sensor_humidity"] <= 100):
    print(f"-Sensor {data['sensor_sensor_index']}- Invalid humidity value {data['sensor_humidity']}. Seting humidity to None.")
    data["sensor_humidity"]= None

  if not (isinstance(data["sensor_pressure"], (int, float)) and 750 <= data["sensor_pressure"] <= 1300):
    print(f"-Sensor {data['sensor_sensor_index']}- Invalid pressure value {data['sensor_pressure']}. Seting pressure to None.")
    data["sensor_pressure"]= None

  if not (isinstance(data["sensor_pm2.5"], (int, float)) and 0 <= data["sensor_pm2.5"] <= 500):
    print(f"-Sensor {data['sensor_sensor_index']}- Invalid pressure value {data['sensor_pm2.5']}. Seting pressure to None.")
    data["sensor_pm2.5"]= None

In [ ]:
#data=get_data(url, headers, params)
data, now_unix = get_data(url, headers, params)
flattened_data = flatten_dict(data)
#print(json.dumps(flattened_data, indent=4))

data_freshness = check_sensor_freshness(flattened_data, now_unix)
print(f"data freshness= {data_freshness}")
if data_freshness == True:
  print("freschness chek passed, cleaning data")
  check_data_quality(flattened_data)

KeyError: 'sensor'

In [ ]:
import requests
import os
import json
import datetime
import logging
import pandas as pd
from datetime import timedelta, timezone

SENSOR_INDEX = YOUR_OWN_INFO
BASE_URL = YOUR_OWN_INFO
API_KEY = YOUR_OWN_INFO

def get_data(url, headers, params):
  try:
    response = requests.get(url, headers=headers, params=params, timeout=15)
    response.raise_for_status()  # Raise an exception for HTTP errors
    data = response.json()
    # get current time after fetching raw data to compare for freshness check
    now = datetime.datetime.now()
    now_unix = int(now.timestamp())
    #repor info and return data and curent time
    logging.info(f"Successfully fetched data from {url}")
    return data, now_unix
  except requests.exceptions.HTTPError as e:
    logging.error(f"HTTP Error fetching data from {url}: {e} - Response: {response.text}")
    return None, now_unix
  except requests.exceptions.ConnectionError as e:
    logging.error(f"Connection Error fetching data from {url}: {e}")
    return None, now_unix
  except requests.exceptions.Timeout as e:
    logging.error(f"Timeout Error fetching data from {url}: {e}")
    return None, now_unix
  except json.JSONDecodeError as e:
    logging.error(f"JSON Decode Error for {url}: {e} - Raw Response: {response.text}")
    return None, now_unix
  except requests.exceptions.RequestException as e:
    logging.error(f"An unexpected Request Error occurred for {url}: {e}")
    return None, now_unix


## check if sensor is working by comparing the last seen timestamp with the curent time and a given delay range set to 10 minutes
def check_sensor_freshness(data, now_unix):

  delay_range=600 #10 minutes in seconds

  if not data:
    logging.warning(f"NO DATA RECIEVED for Sensor_index{SENSOR_INDEX }")
    print("no data recived")
    return False
  if not data["sensor"]['last_seen']:
    logging.warning(f"Sensor data missing 'last_seen' timestamp. Cannot determine freshness.")
    print("Can not determine freshness of data as no sensor last seen data recieved")
    return False

  #convert last seen and now_unix into datetime timestamps for more robust comparison
  last_seen_dt_utc = datetime.fromtimestamp(data["sensor"]['last_seen'], tz=timezone.utc)
  fetch_dt_utc = datetime.fromtimestamp(now_unix, tz=timezone.utc)

  time_difference = fetch_dt_utc - last_seen_dt_utc

  if time_difference > timedelta(seconds=delay_range):
    logging.info(f"Freshness check passed. Last seen {now_unix-data['sensor']['last_seen']}s ago (> {delay_range/60} minute  threshold).")
    print(f"Sensor is working in given delay range {delay_range} seconds")
    return True
  else:
    logging.warning(f"Data is STALE. Last seen {now_unix-data['sensor']['last_seen']}s ago (> {delay_range/60} minute  threshold).")
    print("sensor is stale")
    return False

# Define the flatten_dict function
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


## check validity of Enviromental data
def clean_and_validate_sensor_data(data):


  if not (isinstance(data["sensor_temperature"], (int, float)) and 0 <= data["sensor_temperature"] <= 120):
    print(f"-Sensor {data['sensor_sensor_index']}- Invalid temperature value {data['sensor_temperature']}. Seting Temperatur to None.")
    data["sensor_temperature"]= None

  if not (isinstance(data["sensor_humidity"], (int, float)) and 0 <= data["sensor_humidity"] <= 100):
    print(f"-Sensor {data['sensor_sensor_index']}- Invalid humidity value {data['sensor_humidity']}. Seting humidity to None.")
    data["sensor_humidity"]= None

  if not (isinstance(data["sensor_pressure"], (int, float)) and 750 <= data["sensor_pressure"] <= 1300):
    print(f"-Sensor {data['sensor_sensor_index']}- Invalid pressure value {data['sensor_pressure']}. Seting pressure to None.")
    data["sensor_pressure"]= None

  if not (isinstance(data["sensor_pm2.5"], (int, float)) and 0 <= data["sensor_pm2.5"] <= 500):
    print(f"-Sensor {data['sensor_sensor_index']}- Invalid pressure value {data['sensor_pm2.5']}. Seting pressure to None.")
    data["sensor_pm2.5"]= None

def get_data():
  endpoint = f"/sensors/{SENSOR_INDEX}/"
  url = f"{BASE_URL}{endpoint}"

  headers = {
    "X-API-Key": API_KEY,
    "Accept": "application/json"
    }
  params = {
    "fields": "last_seen,rssi,humidity,temperature,pressure,pm2.5"
    #"fields": "last_seen,rssi,latitude,longitude,humidity,temperature,pressure,pm2.5,pm2.5_alt,visual_range, pm2.5_6hour"
    }

  data, now_unix = get_data(url, headers, params)

  data_freshness = check_sensor_freshness(data, now_unix)
  #print(f"data freshness= {data_freshness}")
  if data_freshness == True:
    flattened_data = flatten_dict(data)
    logging.info("freschness chek passed, cleaning data")
    #print("freschness chek passed, cleaning data")
    data= clean_and_validate_sensor_data(flattened_data)
    print(data["sensor_last_seen"])
    df = pd.DataFrame(data, index=[0]).drop(columns=["api_version","time_stamp","data_time_stamp"], axis=1)
    unix_to_timestamp = datetime.datetime.fromtimestamp(data["sensor_last_seen"])
    df["date"]=unix_to_timestamp.strftime('%Y-%m-%d')
    df["time"]=unix_to_timestamp.strftime('%H:%M:%S')
    return df
  else:
    logging.warning("DATA NOT FRESH. NO Data prepareable")
  #print(f"-------------DEBUGGİNG---------------\n {data} \n {now_unix}")



In [ ]:
df

,sensor_sensor_index,sensor_last_seen,sensor_rssi,sensor_humidity,sensor_temperature,sensor_pressure,sensor_pm2.5,date,time
0,156275,1749559942,-76,34,86,897.9,7.8,2025-06-10,12:52:22


In [ ]:
df=prepare_data()

{'api_version': 'V1.1.0-1.0.41', 'time_stamp': 1749628711, 'data_time_stamp': 1749628704, 'sensor': {'sensor_index': 156275, 'last_seen': 1749628598, 'rssi': -74, 'humidity': 35, 'temperature': 86, 'pressure': 897.92, 'pm2.5': 8.2, 'pm2.5_alt': 5.4, 'visual_range': 128.8, 'stats': {'pm2.5': 8, 'pm2.5_6hour': 7.8, 'time_stamp': 1749628598}}}


In [ ]:
df

,sensor_index,last_seen,rssi,humidity,temperature,pressure,pm2.5,pm2.5_alt,visual_range,pm2.5_6hour,date,time
0,156275,1749628598,-74,35,86,897.92,8.2,5.4,128.8,7.8,2025-06-11,07:56:38


## Fully developt function
process_purpleair_data as ajusted previous get_data dunction
included lambda_handler

In [ ]:
import requests
import os
import json
import datetime
import logging
import pandas as pd
from datetime import datetime, timedelta, timezone

SENSOR_INDEX = os.environ.get("SENSOR_INDEX")
BASE_URL = os.environ.get("BASE_URL")
API_KEY = os.environ.get("API_KEY")




def get_data(url, headers, params):
  try:
    response = requests.get(url, headers=headers, params=params, timeout=15)
    response.raise_for_status()  # Raise an exception for HTTP errors
    data = response.json()
    # get current time after fetching raw data to compare for freshness check
    now = datetime.now(timezone.utc)
    #report info and return data and curent time
    logging.info(f"Successfully fetched data from {url}")
    return data, now
  except requests.exceptions.HTTPError as e:
    logging.error(f"HTTP Error fetching data from {url}: {e} - Response: {getattr(response, 'text', 'No response text available')}")
    return None, now
  except requests.exceptions.ConnectionError as e:
    logging.error(f"Connection Error fetching data from {url}: {e}")
    return None, now
  except requests.exceptions.Timeout as e:
    logging.error(f"Timeout Error fetching data from {url}: {e}")
    return None, now
  except json.JSONDecodeError as e:
    logging.error(f"JSON Decode Error for {url}: {e} - Raw Response: {getattr(response, 'text', 'No response text available')}")
    return None, now
  except requests.exceptions.RequestException as e:
    logging.error(f"An unexpected Request Error occurred for {url}: {e}")
    return None, now


## check if sensor is working by comparing the last seen timestamp with the curent time and a given delay range set to 10 minutes
def check_sensor_freshness(data, now):

  delay_range=600 #10 minutes in seconds

  if not data:
    logging.warning(f"NO DATA RECIEVED for Sensor_index{SENSOR_INDEX }")
    return False
  if not data["sensor"]['last_seen']:
    logging.warning(f"Sensor data missing 'last_seen' timestamp. Cannot determine freshness.")
    return False

  #convert last seen and now_unix into datetime timestamps for more robust comparison
  last_seen_dt_utc = datetime.fromtimestamp(data["sensor"]['last_seen'], tz=timezone.utc)

  time_difference = now - last_seen_dt_utc

  if time_difference < timedelta(seconds=delay_range):
    logging.info(f"Freshness check passed. Last seen {time_difference.total_seconds()}s ago (> {delay_range/60} minute  threshold).")
    return True
  else:
    logging.warning(f"Data is STALE. Last seen {time_difference.total_seconds()}s ago (> {delay_range/60} minute  threshold).")
    return False

## check validity of Enviromental data
def clean_and_validate_sensor_data(data):
  #copy data for safety
  cleaned_data = data.copy()

  def validate_and_update_data(key, min_value, max_value):
    value=cleaned_data.get(key)
    if not (isinstance(value, (int, float))):
      logging.warning(f"-Sensor {cleaned_data['sensor_index']}- {key} is not numeric. Set {key} to None.")
      cleaned_data[key]=None
      return

    if not (min_value <= value <= max_value):
      logging.warning(f"-Sensor {cleaned_data['sensor_index']}- {key} value: {value} out of range. Set {key} to None.")
      cleaned_data[key]=None

    elif isinstance(value, float):
      cleaned_data[key]=round(value, 2)

  validate_and_update_data("temperature", 0, 120)
  validate_and_update_data("humidity", 0, 100)
  validate_and_update_data("pressure", 750, 1300)
  validate_and_update_data("pm2.5", 0, 500)
  validate_and_update_data("pm2.5_alt", 0, 500)
  validate_and_update_data("visual_range", 0, 500)

  return cleaned_data


def process_purpleair_data():
  endpoint = f"/sensors/{SENSOR_INDEX}/"
  url = f"{BASE_URL}{endpoint}"

  headers = {
    "X-API-Key": API_KEY,
    "Accept": "application/json"
    }
  params = {
    "fields": "last_seen,rssi,humidity,temperature,pressure,pm2.5,pm2.5_alt,visual_range, pm2.5_6hour"
    }

  data, now = get_data(url, headers, params)
  data_freshness = check_sensor_freshness(data, now)

  if data_freshness == True:
    logging.info("freschness chek passed, cleaning data")

    #unpack sensor and statts data
    sensor_data = data.get("sensor")
    if sensor_data is None:
      logging.warning(f"API response for sensor {SENSOR_INDEX} did not contain a 'sensor' payload. Cannot proceed.")
      return None

    stats_dict = sensor_data.get('stats')
    if stats_dict and isinstance(stats_dict, dict):
      pm25_6hour_value = stats_dict.get('pm2.5_6hour')
      sensor_data['pm2.5_6hour'] = pm25_6hour_value
    else:
      sensor_data['pm2.5_6hour'] = None
      logging.warning("Stats sub-dictionary not found or is not a dictionary. Setting pm2.5_6hour to None.")
    sensor_data.pop('stats', None)

    sensor_data= clean_and_validate_sensor_data(sensor_data)
    df = pd.DataFrame(sensor_data, index=[0])
    unix_to_timestamp = datetime.fromtimestamp(sensor_data["last_seen"], tz=timezone.utc)
    df["date"]=unix_to_timestamp.strftime('%Y-%m-%d')
    df["time"]=unix_to_timestamp.strftime('%H:%M:%S')
    df['ingestion_timestamp_iso_utc'] = now.isoformat()
    df['ingestion_timestamp_unix_utc'] = int(now.timestamp())
    logging.info(f"Successfully prepared DataFrame for sensor {sensor_data['sensor_index']}.")
    return df

  else:
    logging.warning("DATA NOT FRESH. NO Data prepareable")


def lambda_handler(event, context):
  logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
  logging.info("Lambda function triggered: Starting PurpleAir data ingestion.")

  try:
    # core data processing function
    final_df = process_purpleair_data()

    if final_df is None:
      logging.warning("No data DataFrame was generated. Skipping upload.")
      return {
                'statusCode': 200,
                'body': json.dumps('Data processing completed, but no DataFrame to upload.')
            }

    # Save the DataFrame to AWS S3
    s3_bucket_name = os.environ.get("S3_BUCKET_NAME")
    if not s3_bucket_name:
      logging.error("S3_BUCKET_NAME environment variable not set.")
      return {
                'statusCode': 500,
                'body': json.dumps('Configuration error: S3 bucket name missing.')
            }
    # Define a unique file name for the data in S3 Use ingestion timestamp and sensor ID to ensure uniqueness
    ingestion_time_iso = final_df['ingestion_timestamp_iso_utc'].iloc[0] if 'ingestion_timestamp_iso_utc' in final_df.columns else datetime.now(timezone.utc).isoformat()
    sensor_id_for_file = final_df['globalID'].iloc[0] if 'globalID' in final_df.columns else f"PA_Sensor_{SENSOR_INDEX}"


    s3_key = f"data/purpleair/{datetime.now(timezone.utc).strftime('%Y/%m/%d')}/{sensor_id_for_file}_{ingestion_time_iso.replace(':', '-')}.parquet" # or .csv



    import io
    buffer = io.BytesIO()
    final_df.to_parquet(buffer, index=False)
    buffer.seek(0) # Rewind the buffer to the beginning

    import boto3
    s3_client = boto3.client('s3')
    s3_client.put_object(Bucket=s3_bucket_name, Key=s3_key, Body=buffer.getvalue())

    logging.info(f"Successfully uploaded data for sensor {SENSOR_INDEX} to s3://{s3_bucket_name}/{s3_key}")

    return {
            'statusCode': 200,
            'body': json.dumps('Data ingestion and upload to S3 successful!')
        }

  except Exception as e:
    logging.error(f"An error occurred during Lambda execution: {e}", exc_info=True)
    return {
            'statusCode': 500,
            'body': json.dumps(f'Error during data ingestion: {e}')
        }

## Read paruqet file dowloaded from S3 Bucket to manualy validate the function is properly set up

In [2]:
import pandas as pd
# Assuming you downloaded the file to your current directory
df = pd.read_parquet('PA_Sensor_156275_2025-06-27T06-00-16.978535+00-00.parquet')
print(df.head())

   sensor_index   last_seen  rssi  humidity  temperature  pressure  pm2.5  \
0        156275  1751003883   -69        33           83    901.55   23.1   

   pm2.5_alt  visual_range  pm2.5_6hour        date      time  \
0       13.5          66.2          9.0  2025-06-27  05:58:03   

        ingestion_timestamp_iso_utc  ingestion_timestamp_unix_utc  
0  2025-06-27T06:00:16.978535+00:00                    1751004016  
